# How to use different hosts to execute tasks in the same workflow

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Tasks are usually submitted to queues specified from command line (option `-q`)
  * Tasks with specified queues (`task: queue='name'`) are not affected
  * Different queues can be specified in a SoS workflow

## Default queue

Tasks are by default sent to `localhost`, in which tasks are executed as background processes.

In [4]:
%run -v0

input: for_each=dict(i=range(2))
task:
sh: expand=True
echo "I am task {i}"

,default,Workflow ID 96c43d5f7256320d,Index #4,completed Ran for 4 sec


,6ebf5fda326f8f22,3b7c52cbe3ce2753default,Ran for < 5 seconds,completed


,4fe780b5245b9e7b,3b7c52cbe3ce2753default,Ran for < 5 seconds,completed


You can specify a different queue from the command line, this queue will be used for all tasks in the workflow. In this example I am submitting the tasks to a virtual machine with a [Task Spooler](http://manpages.ubuntu.com/manpages/xenial/man1/tsp.1.html) batch system:

In [5]:
%run -q vm -v0

input: for_each=dict(i=range(2))
task:
sh: expand=True
echo "I am task {i}"

,default,Workflow ID 96c43d5f7256320d,Index #5,completed Ran for 49 sec


,4fe780b5245b9e7b,3b7c52cbe3ce2753default,Ran for < 5 seconds,completed


,6ebf5fda326f8f22,3b7c52cbe3ce2753default,Ran for < 5 seconds,completed


As a special case, you can specify from command line option `-q none`, which basically disables the task statement so that tasks are executed as part of the step process.

In [6]:
%run -q none -v0

input: for_each=dict(i=range(2))
task:
sh: expand=True
echo "I am task {i}"

## Task specific queues

The `default` queue is applied to all tasks in a workflow. If some of your tasks can only be executed in a particular remote host, it makes sense to specify it with the task using the `queue` option. In this case, the command line option will be ignored:

In [11]:
%run -s force

input: for_each=dict(i=range(2))
task: queue='vm'
sh: expand=True
echo "I am task {i}"

,default,Workflow ID 96c43d5f7256320d,Index #11,completed Ran for 6 sec


INFO: Running default :

INFO: 4fe780b5245b9e7b re-execute completed

INFO: 6ebf5fda326f8f22 re-execute completed

INFO: 4fe780b5245b9e7b submitted to vm with job id 425

INFO: 6ebf5fda326f8f22 submitted to vm with job id 426

INFO: Workflow default (ID=96c43d5f7256320d) is executed successfully with 1 completed step, 2 completed substeps and 2 completed tasks.

Actually, the `ignored` description is not entirely correct because the default queue will still be applied to tasks without default queue, so tasks in your workflow can have send to multiple queues. For example, the following workflow would send three tasks to `q0` (default, applied to step `1`), `q1` (applied to step `1`) and `q2` (applied to step `2`).

In [ ]:
%run -q q0 -s force

[1]
task:
print(f'this is task {step_name}')

[2]
task: queue='q1'
print(f'this is task {step_name}')

[3]
task: queue='q2'
print(f'this is task {step_name}')


## Further reading

* 